In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyxona import File
import quantities as pq
import ripple_detection as rd
from scipy.signal import butter, lfilter
import tools

In [2]:
fnames = [
    ['KKL196', 201015, 1],
    ['KKL196', 201015, 2],
    ['KKL196', 201016, 3],
    ['KKL196', 201017, 3],
    ['KKL196', 201018, 2],
    ['KKL196', 201020, 1],  
]

df_fnames = pd.DataFrame(fnames, columns=['animal', 'date', 'session'])


path = "/home/jovyan/work/RippleSpindleCoupling/test_data/"

>For ripple detection (Boyce et al., 2016; Nakashiba et al., 2009), the LFP obtained from CA1
pyramidal cell layer was first band-pass filtered (100–250 Hz), and amplitude was calculated using
the Hilbert transform. Ripple windows were characterized as signals that exceed the amplitude
threshold (three times the standard deviation). Signals that were less than 50 ms apart were merged.

> For spindle detection (Eschenko et al., 2006; Phillips et al., 2012), the LFP obtained from ACC
was band-pass filtered (12–15 Hz), and amplitude was calculated using the Hilbert transform. Spindle
windows were characterized as signals that exceed the amplitude threshold (two times the standard
deviation), with minimum and maximum duration of 200 and 2000 ms, respectively. Signals that are
less than 100 ms apart were merged.

In [3]:
params_ripples = {
    'freq_min': 100,
    'freq_max': 250,
    'cheby1_ord': 4,
    'cheby1_ripple': 0.1,
    'dur_min': 0.01,
    'dur_max': 1.,
    'dur_merge': 0.05,
#    'dur_merge': False,
    'ampl_thresh': 3.,
    't_excl_edge': 10,
    'return_info': 
        {'evts_ts_strtstop' : True,                                                                                                                                                                                                                                                                                                                                                                 
         'evts_ts_center' : True,                                                                                                                                                                                    
         'evts_ampl' : [-4, 4],
         'evts_data' : [-4, 4]}
}
params_spindles = {
    'freq_min': 12,
    'freq_max': 15,
    'cheby1_ord': 4,
    'cheby1_ripple': 0.1,
    'dur_min': 0.2,
    'dur_max': 2.,
    'dur_merge': 0.1,
    'ampl_thresh': 2,
    't_excl_edge': 10,
    'return_info': 
        {'evts_ts_strtstop' : True,                                                                                                                                                                                                                                                                                                                                                                 
         'evts_ts_center' : True,                                                                                                                                                                                    
         'evts_ampl' : [-4, 4],
         'evts_data' : [-4, 4],}
}

In [4]:
ls_df_rppls = []
ls_df_spndls = []

for index, row in df_fnames.iterrows():
    fname = row['animal']+'_'+str(row['date'])+'_'+str(row['session'])+'.set'
    f = File(path+fname)
    lfp_hpc = f.analog_signals[0]
    lfp_ctx = f.analog_signals[1]

    df_rppls_i = tools.event_detection(
        lfp_hpc.signal.magnitude,
        lfp_hpc.sample_rate.magnitude,
        **params_ripples)
        
    df_spndls_i = tools.event_detection(
        lfp_ctx.signal.magnitude,
        lfp_ctx.sample_rate.magnitude,
        **params_spindles)
    
    # merge with df_fnames
    d = row.to_dict()
    df_fnames_i = pd.DataFrame({k: [v] for k, v in d.items()})
    df_fnames_i['tmp'] = 1
    df_fnames['duration'] = f.attrs['duration']
    
    df_rppls_i['tmp'] = 1
    df_rppls_i = pd.merge(df_fnames_i, df_rppls_i, on=['tmp'])
    df_rppls_i.drop('tmp', axis=1, inplace=True)

    df_spndls_i['tmp'] = 1
    df_spndls_i = pd.merge(df_fnames_i, df_spndls_i, on=['tmp'])
    df_spndls_i.drop('tmp', axis=1, inplace=True)

    ls_df_rppls.append(df_rppls_i)
    ls_df_spndls.append(df_spndls_i)
    
df_rppls = pd.concat(ls_df_rppls, ignore_index=True, sort=False)
df_spndls = pd.concat(ls_df_spndls, ignore_index=True, sort=False)

/opt/conda/lib/python3.8/site-packages/pyxona/core.py:589: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.fromfile(f, dtype=sample_dtype, count=sample_count)
/opt/conda/lib/python3.8/site-packages/pyxona/core.py:589: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.fromfile(f, dtype=sample_dtype, count=sample_count)
/opt/conda/lib/python3.8/site-packages/pyxona/core.py:589: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.fromfile(f, dtype=sample_dtype, count=sample_count)
/opt/conda/lib/python3.8/site-packages/pyxona/core.py:589: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a f

In [5]:
df_fnames_i

,animal,date,session,tmp
0,KKL196,201020,1,1


In [6]:
for index, row in df_fnames.iterrows():
    fname = row['animal']+'_'+str(row['date'])+'_'+str(row['session'])+'.set'
    f = File(path+fname)
    lfp_hpc = f.analog_signals[0]
    lfp_ctx = f.analog_signals[1]

/opt/conda/lib/python3.8/site-packages/pyxona/core.py:589: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.fromfile(f, dtype=sample_dtype, count=sample_count)


In [7]:
# TODO:# Visualize ripples, Spindles, make sure you are really computing the right thing.
f.attrs

{'trial_date': 'Saturday, 14 Dec 2019',
 'trial_time': '06:03:36',
 'experimenter': None,
 'comments': None,
 'duration': 1800,
 'sw_version': '1.2.2.7',
 'ADC_fullscale_mv': 1500,
 'tracker_version': 0,
 'stim_version': 1,
 'audio_version': 0,
 'gain_ch_0': 1500,
 'filter_ch_0': 3,
 'a_in_ch_0': 0,
 'b_in_ch_0': 4,
 'mode_ch_0': 0,
 'filtresp_ch_0': 0,
 'filtkind_ch_0': 0,
 'filtfreq1_ch_0': 0,
 'filtfreq2_ch_0': 0,
 'filtripple_ch_0': 0.0,
 'filtdcblock_ch_0': 0,
 'dispmode_ch_0': 1,
 'channame_ch_0': None,
 'gain_ch_1': 1500,
 'filter_ch_1': 3,
 'a_in_ch_1': 1,
 'b_in_ch_1': 4,
 'mode_ch_1': 0,
 'filtresp_ch_1': 0,
 'filtkind_ch_1': 0,
 'filtfreq1_ch_1': 0,
 'filtfreq2_ch_1': 0,
 'filtripple_ch_1': 0.0,
 'filtdcblock_ch_1': 0,
 'dispmode_ch_1': 1,
 'channame_ch_1': None,
 'gain_ch_2': 1500,
 'filter_ch_2': 3,
 'a_in_ch_2': 2,
 'b_in_ch_2': 4,
 'mode_ch_2': 0,
 'filtresp_ch_2': 0,
 'filtkind_ch_2': 0,
 'filtfreq1_ch_2': 0,
 'filtfreq2_ch_2': 0,
 'filtripple_ch_2': 0.0,
 'filtdcblock_

### Event rate by animal

In [8]:
# We take last ripple event as timepoint when recording stops. Imperfect, but okay for now.
t_trial = df_rppls.groupby(['animal', 'date', 'session'])['evts_ts_center'].max()
t_trial

animal  date    session
KKL196  201015  1          1716.733958
                2          1767.522500
        201016  3          1788.968854
        201017  3          1789.794063
        201018  2          1789.443229
        201020  1          1788.675938
Name: evts_ts_center, dtype: float64

In [9]:
n_rppls_trial = df_rppls.groupby(['animal', 'date', 'session']).size()
n_rppls_trial

animal  date    session
KKL196  201015  1          1026
                2          1221
        201016  3          1226
        201017  3          1352
        201018  2          1241
        201020  1          1116
dtype: int64

In [10]:
n_spndls_trial = df_spndls.groupby(['animal', 'date', 'session']).size()
n_spndls_trial

animal  date    session
KKL196  201015  1          127
                2          102
        201016  3          171
        201017  3          116
        201018  2          181
        201020  1          115
dtype: int64

In [11]:
n_rppls_trial/t_trial

animal  date    session
KKL196  201015  1          0.597646
                2          0.690797
        201016  3          0.685311
        201017  3          0.755394
        201018  2          0.693512
        201020  1          0.623925
dtype: float64

In [12]:
n_spndls_trial/t_trial

animal  date    session
KKL196  201015  1          0.073978
                2          0.057708
        201016  3          0.095586
        201017  3          0.064812
        201018  2          0.101149
        201020  1          0.064293
dtype: float64

In [13]:
a = np.logical_and(df_spndls['evts_ts_center']>1, df_spndls['evts_ts_center']<1790)

In [14]:
df_rppls

,animal,date,session,index,evts_ts_strtstop,evts_ts_center,evts_ampl,evts_ampl_ts,evts_data,evts_data_ts
0,KKL196,201015,1,2,"(22.980208333333334, 23.040625000000002)",23.010417,"[48.4321287758511, 46.22964071002017, 43.70719...","[18.980208333333334, 18.980416666666667, 18.98...","[-182.1441650390625, -176.1474609375, -170.516...","[18.980208333333334, 18.980416666666667, 18.98..."
1,KKL196,201015,1,3,"(29.147916666666667, 29.198541666666667)",29.173229,"[6.187054609542445, 7.9979137961775635, 11.004...","[25.147916666666667, 25.148125, 25.14833333333...","[50.6744384765625, 29.84619140625, 4.348754882...","[25.147916666666667, 25.148125, 25.14833333333..."
2,KKL196,201015,1,4,"(45.620625000000004, 45.63979166666667)",45.630208,"[5.412582167913908, 5.350152387810702, 5.69666...","[41.620625000000004, 41.62083333333334, 41.621...","[-111.8316650390625, -113.067626953125, -117.4...","[41.620625000000004, 41.62083333333334, 41.621..."
3,KKL196,201015,1,5,"(45.916250000000005, 45.92979166666667)",45.923021,"[31.010726363652825, 30.878200187871684, 30.78...","[41.916250000000005, 41.91645833333334, 41.916...","[157.2418212890625, 150.87890625, 150.37536621...","[41.916250000000005, 41.91645833333334, 41.916..."
4,KKL196,201015,1,6,"(71.81375, 72.04958333333333)",71.931667,"[30.442992073553338, 28.538233080608148, 26.55...","[67.81375, 67.81395833333333, 67.8141666666666...","[415.10009765625, 411.5753173828125, 409.69848...","[67.81375, 67.81395833333333, 67.8141666666666..."
...,...,...,...,...,...,...,...,...,...,...
7177,KKL196,201020,1,1112,"(1781.616041666667, 1781.6735416666668)",1781.644792,"[48.770170552459305, 48.18115197919928, 47.519...","[1777.616041666667, 1777.61625, 1777.616458333...","[-160.400390625, -166.56494140625, -171.691894...","[1777.616041666667, 1777.61625, 1777.616458333..."
7178,KKL196,201020,1,1113,"(1782.6027083333333, 1782.613125)",1782.607917,"[40.69967118189174, 41.54359325547703, 42.2257...","[1778.6027083333333, 1778.6029166666667, 1778....","[-358.88671875, -354.55322265625, -350.4638671...","[1778.6027083333333, 1778.6029166666667, 1778...."
7179,KKL196,201020,1,1114,"(1786.0135416666667, 1786.053541666667)",1786.033542,"[42.90615343100634, 43.99216566875423, 45.3311...","[1782.0135416666667, 1782.01375, 1782.01395833...","[-447.57080078125, -446.47216796875, -441.4672...","[1782.0135416666667, 1782.01375, 1782.01395833..."
7180,KKL196,201020,1,1115,"(1787.0131250000002, 1787.0425)",1787.027812,"[5.28677846854825, 5.778135519853612, 6.506965...","[1783.0131250000002, 1783.0133333333335, 1783....","[73.2421875, 74.0966796875, 73.6083984375, 71....","[1783.0131250000002, 1783.0133333333335, 1783...."


In [15]:
df_spndls

,animal,date,session,index,evts_ts_strtstop,evts_ts_center,evts_ampl,evts_ampl_ts,evts_data,evts_data_ts
0,KKL196,201015,1,4,"(12.116041666666668, 12.3925)",12.254271,"[14.975701125549648, 14.97916620643128, 14.982...","[8.116041666666668, 8.11625, 8.116458333333334...","[-55.023193359375, -51.61285400390625, -43.304...","[8.116041666666668, 8.11625, 8.116458333333334..."
1,KKL196,201015,1,5,"(16.8325, 17.526666666666667)",17.179583,"[8.090745939681362, 8.106585232513844, 8.12241...","[12.832500000000001, 12.832708333333334, 12.83...","[-109.7259521484375, -107.1624755859375, -104....","[12.832500000000001, 12.832708333333334, 12.83..."
2,KKL196,201015,1,6,"(18.597291666666667, 18.843333333333334)",18.720312,"[7.054343541062104, 7.050011883436324, 7.04571...","[14.597291666666667, 14.5975, 14.5977083333333...","[-126.617431640625, -126.9378662109375, -123.5...","[14.597291666666667, 14.5975, 14.5977083333333..."
3,KKL196,201015,1,7,"(20.950625000000002, 21.655833333333334)",21.303229,"[25.55137459730074, 25.545221692167942, 25.538...","[16.950625000000002, 16.950833333333335, 16.95...","[-84.5947265625, -92.559814453125, -101.188659...","[16.950625000000002, 16.950833333333335, 16.95..."
4,KKL196,201015,1,8,"(23.562083333333334, 23.79354166666667)",23.677813,"[10.518992508364825, 10.529264132068416, 10.53...","[19.562083333333334, 19.562291666666667, 19.56...","[31.05926513671875, 27.6947021484375, 24.37591...","[19.562083333333334, 19.562291666666667, 19.56..."
...,...,...,...,...,...,...,...,...,...,...
807,KKL196,201020,1,110,"(1647.7325, 1647.9841666666669)",1647.858333,"[8.444516875924927, 8.438715639772196, 8.43289...","[1643.7325, 1643.7327083333334, 1643.732916666...","[-89.05792236328125, -81.1614990234375, -73.40...","[1643.7325, 1643.7327083333334, 1643.732916666..."
808,KKL196,201020,1,111,"(1669.7908333333335, 1670.0204166666667)",1669.905625,"[10.39007714221933, 10.396856350099172, 10.403...","[1665.7908333333335, 1665.7910416666668, 1665....","[-10.162353515625, -15.49530029296875, -21.972...","[1665.7908333333335, 1665.7910416666668, 1665...."
809,KKL196,201020,1,112,"(1695.7310416666667, 1696.0085416666668)",1695.869792,"[8.40460351062212, 8.4032575887828, 8.40191785...","[1691.7310416666667, 1691.73125, 1691.73145833...","[-41.86248779296875, -45.8221435546875, -46.62...","[1691.7310416666667, 1691.73125, 1691.73145833..."
810,KKL196,201020,1,113,"(1702.9193750000002, 1703.206875)",1703.063125,"[5.259911119908886, 5.2737269089495955, 5.2875...","[1698.9193750000002, 1698.9195833333335, 1698....","[-52.4139404296875, -48.5687255859375, -44.357...","[1698.9193750000002, 1698.9195833333335, 1698...."
